[View in Colaboratory](https://colab.research.google.com/github/adowaconan/Deep_learning_fMRI/blob/master/2_full_connected_neural_network.ipynb)

# standard way of uploading data from your google drive to the colab working space

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import numpy as np
import zipfile
data_id = {}
data_id['3439'] = '1Y9NTSNs9sHoIa6DnGpPoq41yGWn4fkE6'#
sub = '3439'
file_id = data_id[sub]
import os
if not os.path.exists('/content/'+'sub_%s'%sub):
    # import data from google drive to the working machine
    zip_import = drive.CreateFile({'id':file_id})
    zip_import.GetContentFile('sub_%s'%sub)
    # unzip the file
    zip_ref = zipfile.ZipFile('sub_%s'%sub,'r')
    zip_ref.extractall()
    zip_ref.close()

# instand python libraries using pip-install

In [4]:
!pip install tqdm
!apt-get install swig
!pip install -U pymvpa2

    100% |████████████████████████████████| 51kB 3.6MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 0 not upgraded.
Need to get 1,080 kB of archives.
After this operation, 5,657 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/universe amd64 swig3.0 amd64 3.0.10-1.2 [1,074 kB]
Get:2 http://archive.ubuntu.com/ubuntu artful/universe amd64 swig amd64 3.0.10-1.2 [6,384 B]
Fetched 1,080 kB in 1s (796 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.10-1.2_amd64.deb ...
Unpacking swig3.0 (3.0.10-1.2) ...
Selecting previously unselected package swig

Successfully built pymvpa2 nibabel


# as you see, keras in colab uses tensorflow as default backend

In [5]:
from glob import glob
from tqdm import tqdm
import pickle
from collections import Counter
from mvpa2.mappers.fx import mean_group_sample
from mvpa2.generators.partition import NFoldPartitioner
from mvpa2.base.node import ChainNode
from mvpa2.generators.resampling import Balancer
from mvpa2.generators.base import Sifter

from keras.layers import Conv3D,Dense,Dropout,BatchNormalization
from keras.layers import Input, AveragePooling3D
from keras.layers import Flatten,Dense
from keras.models import Model
import keras
from keras.callbacks import ModelCheckpoint,TensorBoard
from sklearn import metrics
import pandas as pd
import os
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


# Example data - fMRI ROI dataset

1. when the subjects are in the scanner, they read or think deeper when a word is presented
2. regions of interest are extracted and vectorized


# load the data and average the volumns, taking the condition of "reenact"

In [0]:
pr_ds = pickle.load(open('/content/3439/roi_1_bin.pkl','rb'))
pr_ds = pr_ds.get_mapped(mean_group_sample(['chunks','trials'],order='occurrence'))
# get the chosen condition
pr_ds = pr_ds[pr_ds.sa.context == 'reenact']

In [14]:
print(pr_ds.summary())

Dataset: 144x2336@float32, <sa: categories,chunks,context,sample_indices,targets,time_coords,time_indices,trials,words>, <fa: voxel_indices>, <a: imgaffine,imghdr,imgtype,mapper,voxel_dim,voxel_e...
stats: mean=0.00086342 std=0.549984 var=0.302482 min=-2.8908 max=2.82391

Counts of targets in each chunk:
  chunks\targets animal tool
                   ---   ---
        1          18    18
        3          18    18
        5          18    18
        7          18    18

Summary for targets across chunks
  targets mean std min max #chunks
  animal   18   0   18  18    4
   tool    18   0   18  18    4

Summary for chunks across targets
  chunks mean std min max #targets
    1     18   0   18  18     2
    3     18   0   18  18     2
    5     18   0   18  18     2
    7     18   0   18  18     2
Sequence statistics for 144 entries from set ['animal', 'tool']
Counter-balance table for orders up to 2:
Targets/Order O1     |  O2     |
   animal:    36 35  |  32 38  |
    tool:     35 37 

In [0]:
from keras.layers import Input,Dense,Activation
from keras.models import Sequential,Model
from keras import optimizers,losses,metrics

# a toy model of using only fully connected layers

In [31]:
model = Sequential()
model.add(Dense(32, input_shape=(2336,)))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 32)                74784     
_________________________________________________________________
activation_7 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 66        
_________________________________________________________________
activation_8 (Activation)    (None, 2)                 0         
Total params: 74,850
Trainable params: 74,850
Non-trainable params: 0
_________________________________________________________________


# add an optimizer and a loss function before your training

In [0]:
model.compile(optimizer=optimizers.Adam(),loss=losses.binary_crossentropy,metrics=['categorical_accuracy'])

# tensorflow takes tensor matrices or numpy array, thus, we need to convert the data to numpy  array with float32 or float64. The float number conversion is tricky and we will talk about it later

In [33]:
data = pr_ds.samples
label_map = {'animal':[0,1],'tool':[1,0]}
labels = np.array([label_map[item] for item in pr_ds.targets])
data,labels

(array([[ 0.28787753,  0.52017474,  0.23097952, ...,  0.54967576,
          0.42195216,  0.09478402],
        [ 0.0090575 , -0.20558704, -0.10110297, ..., -0.24365346,
         -0.18717152, -0.2724901 ],
        [ 0.77175385,  0.33383298,  0.8632676 , ...,  0.59843194,
         -0.33976305, -0.21052049],
        ...,
        [ 0.6065182 , -0.3108139 , -0.240719  , ...,  0.38944736,
          0.51658106,  0.04471493],
        [-0.60012674, -0.43406656, -0.2635417 , ..., -0.40048814,
         -0.37045285,  0.09566599],
        [-0.27681738, -0.69222397, -0.56372875, ...,  0.25518915,
         -0.17328219, -0.33957395]], dtype=float32), array([[0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
       

# call "fit" function to fit the data. Here, we did not do a cross validation

In [34]:
model.fit(data,labels,batch_size=32,epochs=5,validation_split=0.2,)

Train on 115 samples, validate on 29 samples
Epoch 1/5
115/115 [==============================] - 0s 2ms/step - loss: 0.6768 - categorical_accuracy: 0.5826 - val_loss: 0.5944 - val_categorical_accuracy: 0.7241
Epoch 2/5
115/115 [==============================] - 0s 216us/step - loss: 0.2489 - categorical_accuracy: 0.8870 - val_loss: 0.5429 - val_categorical_accuracy: 0.7241
Epoch 3/5
115/115 [==============================] - 0s 181us/step - loss: 0.1304 - categorical_accuracy: 0.9652 - val_loss: 0.5472 - val_categorical_accuracy: 0.7586
Epoch 4/5
115/115 [==============================] - 0s 176us/step - loss: 0.0707 - categorical_accuracy: 1.0000 - val_loss: 0.5554 - val_categorical_accuracy: 0.7931
Epoch 5/5
115/115 [==============================] - 0s 188us/step - loss: 0.0442 - categorical_accuracy: 1.0000 - val_loss: 0.5650 - val_categorical_accuracy: 0.7931
